In [1]:
import os

import tensorflow as tf

/home/sinho/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('data/fashion_train.csv')

In [6]:
df_train_X = df_train.iloc[:, 1:]
df_train_y = df_train.iloc[:,[0]]

In [7]:
df_train_X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,3,...,23,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,141,73,0,0,107,63,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,35,7,14,0,0,0


In [8]:
train_X = df_train_X

In [9]:
train_y = pd.get_dummies(df_train_y,columns=['label'],sparse=True)

In [10]:
df_test = pd.read_csv('data/fashion_test.csv')

In [11]:
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,1,0,0,0,0,0,0,0,0,0,...,208,174,128,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,1,0,1,...,49,40,3,0,40,66,11,0,0,0
2,8,0,0,0,0,0,0,0,0,0,...,128,126,126,120,129,75,0,0,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,123,145,152,206,58,0,0,0,0,0


In [12]:
df_test_X = df_test.iloc[:,1:]
df_test_y = df_test.iloc[:,[0]]

In [13]:
test_X = df_test_X
test_y = pd.get_dummies(df_test_y, columns=['label'], sparse = True)

ref)https://www.quora.com/What-is-the-best-way-to-read-data-into-Tensorflow

In [14]:
import os

fashion_train = pd.read_csv("./data/fashion_train.csv").values
fashion_test = pd.read_csv("./data/fashion_test.csv").values

In [15]:
def make_TFRecordWriter(csv, name):
    with tf.python_io.TFRecordWriter("{}.tfrecord".format(name)) as tf_writer:
        for row in csv:
            #file formate : feature 1…..feature n, label
            features, label = row[-1:], row[0]
            example = tf.train.Example()
            example.features.feature["features"].float_list.value.extend(features)
            example.features.feature["label"].int64_list.value.append(label)
            tf_writer.write(example.SerializeToString())

In [16]:
make_TFRecordWriter(fashion_train, 'fashion_train')
make_TFRecordWriter(fashion_test, 'fashion_test')

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

fashion_mnist = input_data.read_data_sets('input/data', one_hot=True)

/home/sinho/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting input/data/train-images-idx3-ubyte.gz
Extracting input/data/train-labels-idx1-ubyte.gz
Extracting input/data/t10k-images-idx3-ubyte.gz
Extracting input/data/t10k-labels-idx1-ubyte.gz


In [2]:

# Shapes of training set
print("Training set (images) shape: {shape}".format(shape=fashion_mnist.train.images.shape))
print("Training set (labels) shape: {shape}".format(shape=fashion_mnist.train.labels.shape))

# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=fashion_mnist.test.images.shape))
print("Test set (labels) shape: {shape}".format(shape=fashion_mnist.test.labels.shape))

Training set (images) shape: (55000, 784)
Training set (labels) shape: (55000, 10)
Test set (images) shape: (10000, 784)
Test set (labels) shape: (10000, 10)


## Dynamic_RNN

### basic cell

In [7]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
#rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
#rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

#outputs, states = tf.nn.static_rnn(rnn_basic_cell, X, dtype=tf.float32)
outputs, states = tf.nn.dynamic_rnn(rnn_basic_cell, X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

Epoch:  0001  Avg. cost =  0.4988
Epoch:  0002  Avg. cost =  0.2284
Epoch:  0003  Avg. cost =  0.1850
Epoch:  0004  Avg. cost =  0.1605
Epoch:  0005  Avg. cost =  0.1451
Epoch:  0006  Avg. cost =  0.1298
Epoch:  0007  Avg. cost =  0.1224
Epoch:  0008  Avg. cost =  0.1129
Epoch:  0009  Avg. cost =  0.1138
Epoch:  0010  Avg. cost =  0.1056
Epoch:  0011  Avg. cost =  0.1046
Epoch:  0012  Avg. cost =  0.1002
Epoch:  0013  Avg. cost =  0.0945
Epoch:  0014  Avg. cost =  0.0862
Epoch:  0015  Avg. cost =  0.0881
Epoch:  0016  Avg. cost =  0.0912
Epoch:  0017  Avg. cost =  0.0864
Epoch:  0018  Avg. cost =  0.0819
Epoch:  0019  Avg. cost =  0.0805
Epoch:  0020  Avg. cost =  0.0754
Epoch:  0021  Avg. cost =  0.0777
Epoch:  0022  Avg. cost =  0.0792
Epoch:  0023  Avg. cost =  0.0781
Epoch:  0024  Avg. cost =  0.0804
Epoch:  0025  Avg. cost =  0.0711
Epoch:  0026  Avg. cost =  0.0669
Epoch:  0027  Avg. cost =  0.0683
Epoch:  0028  Avg. cost =  0.0718
Epoch:  0029  Avg. cost =  0.0677
Epoch:  0030  

### LSTM

In [9]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
#rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

#outputs, states = tf.nn.static_rnn(cell=rnn_basic_cell, inputs = X, dtype=tf.float32)
outputs, states = tf.nn.dynamic_rnn(rnn_LSTM_cell, X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

Epoch:  0001  Avg. cost =  0.4118
Epoch:  0002  Avg. cost =  0.1283
Epoch:  0003  Avg. cost =  0.0890
Epoch:  0004  Avg. cost =  0.0674
Epoch:  0005  Avg. cost =  0.0543
Epoch:  0006  Avg. cost =  0.0486
Epoch:  0007  Avg. cost =  0.0405
Epoch:  0008  Avg. cost =  0.0357
Epoch:  0009  Avg. cost =  0.0335
Epoch:  0010  Avg. cost =  0.0266
Epoch:  0011  Avg. cost =  0.0266
Epoch:  0012  Avg. cost =  0.0230
Epoch:  0013  Avg. cost =  0.0226
Epoch:  0014  Avg. cost =  0.0197
Epoch:  0015  Avg. cost =  0.0192
Epoch:  0016  Avg. cost =  0.0161
Epoch:  0017  Avg. cost =  0.0149
Epoch:  0018  Avg. cost =  0.0143
Epoch:  0019  Avg. cost =  0.0127
Epoch:  0020  Avg. cost =  0.0123
Epoch:  0021  Avg. cost =  0.0115
Epoch:  0022  Avg. cost =  0.0108
Epoch:  0023  Avg. cost =  0.0102
Epoch:  0024  Avg. cost =  0.0101
Epoch:  0025  Avg. cost =  0.0113
Epoch:  0026  Avg. cost =  0.0082
Epoch:  0027  Avg. cost =  0.0067
Epoch:  0028  Avg. cost =  0.0085
Epoch:  0029  Avg. cost =  0.0067
Epoch:  0030  

### GRU

In [11]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
#rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

#outputs, states = tf.nn.static_rnn(cell=rnn_GRU_cell, inputs = X, dtype=tf.float32)
outputs, states = tf.nn.dynamic_rnn(rnn_GRU_cell,  X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

Epoch:  0001  Avg. cost =  0.4009
Epoch:  0002  Avg. cost =  0.1065
Epoch:  0003  Avg. cost =  0.0715
Epoch:  0004  Avg. cost =  0.0530
Epoch:  0005  Avg. cost =  0.0432
Epoch:  0006  Avg. cost =  0.0376
Epoch:  0007  Avg. cost =  0.0325
Epoch:  0008  Avg. cost =  0.0271
Epoch:  0009  Avg. cost =  0.0253
Epoch:  0010  Avg. cost =  0.0242
Epoch:  0011  Avg. cost =  0.0192
Epoch:  0012  Avg. cost =  0.0186
Epoch:  0013  Avg. cost =  0.0162
Epoch:  0014  Avg. cost =  0.0184
Epoch:  0015  Avg. cost =  0.0117
Epoch:  0016  Avg. cost =  0.0141
Epoch:  0017  Avg. cost =  0.0123
Epoch:  0018  Avg. cost =  0.0112
Epoch:  0019  Avg. cost =  0.0102
Epoch:  0020  Avg. cost =  0.0099
Epoch:  0021  Avg. cost =  0.0088
Epoch:  0022  Avg. cost =  0.0081
Epoch:  0023  Avg. cost =  0.0073
Epoch:  0024  Avg. cost =  0.0067
Epoch:  0025  Avg. cost =  0.0077
Epoch:  0026  Avg. cost =  0.0085
Epoch:  0027  Avg. cost =  0.0066
Epoch:  0028  Avg. cost =  0.0048
Epoch:  0029  Avg. cost =  0.0058
Epoch:  0030  

## Static_RNN

### basic cell

In [2]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
#rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
#rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

outputs, states = tf.nn.static_rnn(rnn_basic_cell, X, dtype=tf.float32)
#outputs, states = tf.nn.dynamic_rnn(cell = rnn_cell, input = X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

TypeError: inputs must be a sequence

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

NameError: name 'optimizer' is not defined

### LSTM

In [14]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
#rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

outputs, states = tf.nn.static_rnn(rnn_LSTM_cell, X, dtype=tf.float32)
#outputs, states = tf.nn.dynamic_rnn(cell = rnn_cell, input = X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

TypeError: inputs must be a sequence

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_6' with dtype float and shape [?,28,28]
	 [[Node: Placeholder_6 = Placeholder[dtype=DT_FLOAT, shape=[?,28,28], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: Mean_4/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_839_Mean_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_6', defined at:
  File "/home/sinho/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sinho/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sinho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sinho/anaconda3/lib/python3

### GRU

In [16]:
# option setting
#______________
learning_rate = 0.001
total_epoch = 30
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


# neural network model
#____________________
X = tf.placeholder(tf.float32, [None, n_step , n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#rnn_basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
#rnn_LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
rnn_GRU_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

outputs, states = tf.nn.static_rnn(rnn_GRU_cell, X, dtype=tf.float32)
#outputs, states = tf.nn.dynamic_rnn(cell = rnn_cell, input = X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

SyntaxError: positional argument follows keyword argument (<ipython-input-16-aea7bdf4e926>, line 24)

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size, n_step, n_input)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    
    test_batch_size = len(fashion_mnist.test.images)
    test_xs = fashion_mnist.test.images.reshape([test_batch_size, n_step, n_input])
    test_ys = fashion_mnist.test.labels
    
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: test_xs,
                                                                          Y: test_ys}))

NameError: name 'optimizer' is not defined